In [ ]:
!git clone https://github.com/oKatanaaa/GCT

In [ ]:
!pip install --upgrade git+https://github.com/oKatanaaa/MakiFlow.git

In [ ]:
%tensorflow_version 1.x

In [ ]:
from GCT.tools import load_cifar10
import matplotlib.pyplot as plt
%matplotlib inline
from makiflow.models import Classificator
from makiflow.generators.classification import cycle_generator
import tensorflow as tf
from makiflow.layers import *

In [ ]:
(Xtrain, Ytrain), (Xtest, Ytest), (X_mean, X_std) = load_cifar10()

In [ ]:
BATCH_SIZE = 64

In [ ]:
def block(x, id, in_f, out_f):
    x_skip = x
    x_main = x
    x_main = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main{id}_1')(x_main)
    x_main = BatchNormLayer(D=out_f, name=f'bn_main{id}_1')(x_main)
    x_main = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main{id}_2')(x_main)
    x_main = BatchNormLayer(D=out_f, name=f'bn_main{id}_2')(x_main)
    x_main = MaxPoolLayer(name=f'mp{id}')(x_main)

    x_skip = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, stride=2, activation=None, name=f'conv_skip{id}')(x_skip)
    x_skip = BatchNormLayer(D=out_f, name=f'bn_skip{id}_1')(x_skip)

    x_sum = SumLayer(name=f'sum{id}')([x_main, x_skip])
    return x_sum

In [ ]:
inp = InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')
x = block(inp, id=1, in_f=3, out_f=32)
# 16x16
x = block(x, id=2, in_f=32, out_f=64)
# 8x8
x = block(x, id=3, in_f=64, out_f=128)
# 4x4
x = block(x, id=4, in_f=128, out_f=256)
# 2x2
x = FlattenLayer('flat')(x)
# 4*256 = 1024
x = DenseLayer(in_d=1024, out_d=256, name='projection_head')(x)
x = DenseLayer(in_d=256, out_d=10, activation=None, name='classification_head')(x)

In [ ]:
model = Classificator(input=inp, output=x)
sess = tf.Session()
model.set_session(sess)

In [ ]:
opt = tf.train.AdamOptimizer()

In [ ]:
gen = cycle_generator(Xtrain, Ytrain.reshape(-1), BATCH_SIZE)

In [ ]:
info1 = model.genfit_ce(gen, (Xtest, Ytest.reshape(-1)), optimizer=opt, epochs=5, iter=100)

In [ ]:
plt.plot(info1['test errors'])